In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers.trainer_utils import EvaluationStrategy
import os
import torch

In [2]:
df = pd.read_csv("/content/prufrock_metaphor_conversation_final.csv")

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)


In [6]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [9]:
os.environ["WANDB_DISABLED"] = "true"


In [12]:
training_args = TrainingArguments(
    output_dir="./models/prufrock_roberta",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
trainer.train()
eval_results = trainer.evaluate()
print("Evaluation:", eval_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.714000,0.737298,0.307692,0.000000,0.000000,0.000000
2,0.695700,0.696351,0.307692,0.000000,0.000000,0.000000
3,0.672900,0.678857,0.692308,0.750000,0.857143,0.666667


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation: {'eval_loss': 0.6788573861122131, 'eval_accuracy': 0.6923076923076923, 'eval_f1': 0.75, 'eval_precision': 0.8571428571428571, 'eval_recall': 0.6666666666666666, 'eval_runtime': 0.0422, 'eval_samples_per_second': 308.19, 'eval_steps_per_second': 94.828, 'epoch': 3.0}


In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-14-2354326374.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.save_model("./models/prufrock_roberta")


In [25]:
model.eval()

def predict_sentences(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    # Move inputs to the same device as the model
    device = model.device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
    for sent, pred in zip(sentences, predictions):
        print(f"Sentence: {sent}\n→ Prediction: {'Metaphor' if pred == 1 else 'Literal'}\n")

In [28]:
test_sentences = [
    "I have measured out my life with coffee spoons.",
    "He submitted the assignment on time.",
    "i am feeling heavy.",
    "life is a cycle."
]
predict_sentences(test_sentences)

Sentence: I have measured out my life with coffee spoons.
→ Prediction: Metaphor

Sentence: He submitted the assignment on time.
→ Prediction: Literal

Sentence: i am feeling heavy.
→ Prediction: Metaphor

Sentence: life is a cycle.
→ Prediction: Metaphor

